In [1]:
import pandas as pd
import numpy as np

# Data Visualizations
import matplotlib.pyplot as plt
import seaborn as sns


# Data Acquisition
import requests


# Needed for Working with Time Series Data
from vega_datasets import data

In [2]:
# Setting up for function
endpoint = 'sales'
host = 'https://python.zgulde.net/'
api = 'api/v1/'

url = host + api + endpoint

response = requests.get(url)

In [3]:
# Dry Out Function (Dont Repeat Yourself)
# We often build functions once we know that the code we want is 
# reusable and not bespoke
# function that takes in the string "sales", "items", or "stores"

def get_all(endpoint):
    """ Read all records on all pages """
    
    if endpoint not in ["sales", "items", "stores"]:
        return "Not available from this API. Check the documentation"
    
    host = "https://python.zgulde.net/"
    api = "api/v1/"

    url = host + api + endpoint

    response = requests.get(url)

    if response.ok:
        payload = response.json()["payload"]

        # endpoint should be "items", "sales", or "stores"
        contents = payload[endpoint]

        # Make a dataframe of the contents
        df = pd.DataFrame(contents)

        next_page = payload["next_page"]

        while next_page:
            # Append the next_page url piece
            url = host + next_page
            response = requests.get(url)

            payload = response.json()["payload"]

            next_page = payload["next_page"]    
            contents = payload[endpoint]

            df = pd.concat([df, pd.DataFrame(contents)])

            df = df.reset_index(drop=True)

    return df

# Data Acquisition exercises



The end result of this exercise should be a file named acquire.py.

1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.


In [4]:
# Long way
response = requests.get('https://python.zgulde.net/api/v1/items')
data = response.json()
items = pd.DataFrame(data['payload']['items'])

In [5]:
# DRY way
items = get_all("items")
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


2. Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [6]:
# Long way 
response = requests.get('https://api.data.codeup.com/api/v1/stores')
data = response.json()
stores = pd.DataFrame(data['payload']['stores'])

In [7]:
stores = get_all('stores')
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


3. Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [ ]:
sales = get_all("sales")
sales.head()

4. Save the data in your files to local csv files so that it will be faster to access in the future.


In [ ]:
items.to_csv('items.csv')
stores.to_csv('stores.csv')
sales.to_csv('sales.csv')

In [ ]:
sales_and_stores = pd.merge(sales, stores, how = 'inner,
                            left_on = 'store',
                           right_on = 'store_id')

sales_and_stores.head()

5. Combine the data from your three separate dataframes into one large dataframe.

In [ ]:
# Merge everything together
everything = pd.merge(sales_and_stores,
                      items,
                      how="inner",
                      left_on="item",
                      right_on="item_id")
everything.head()

6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [ ]:
electricity_consumption = pd.read_csv("https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv")
electricity_consumption.head()

7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.